In [ ]:
!pip install pandas numpy PyPDF2 nltk pdfplumber

In [94]:
from google.colab import drive
drive.mount('/content/drive')
#folder_path = "/content/drive/MyDrive/DS_PROJ_MAST90106/data/pdf files"
folder_path = "/content/drive/MyDrive/DS_PROJ_MAST90106/data/Cleaned txt data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
import re
import nltk
import os
# import pdfplumber
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
from collections import Counter
from collections import defaultdict
import pandas as pd

In [5]:
def extract(input_string):    
    before_pattern = r'((?:\b[A-Za-z0-9\-\.]+\b\s+){0,5})(\b[A-Za-z0-9\-\.]+)(?=\s+v\b)'
    after_pattern = r'\bv\s+((?:\b[A-Za-z0-9\-\.]+\b\s+){0,5})(\b[A-Za-z0-9\-\.]+)'
    before_match = re.search(before_pattern, input_string, re.IGNORECASE)
    after_match = re.search(after_pattern, input_string, re.IGNORECASE)
    before_words = []
    after_words = []
    if before_match:
        before_words = before_match.group(1).split()
        before_words.append(before_match.group(2))
    if after_match:
        after_words = after_match.group(1).split()
        after_words.append(after_match.group(2))

    if "CITATION" not in before_words:
      return (before_words, after_words)
    else:
      return 0

def clean(input)->list:
  remove_string= ["APPLICANT","PTY","Ltd","Pty","LTD","RESPONDENT","APPLICANTS",
                  "Construction",]
  if type(input) == str:
    input = input.split()
  tmp = []
  for word in input:
    if word not in remove_string:
      tmp.append(word)
  return tmp

In [96]:
df = pd.read_excel('/content/drive/MyDrive/DS_PROJ_MAST90106/Data_checks/rbe_case_info_extraction.xlsx')
df["refer_file"] = None
df["refer_ID"] = None
df = df.fillna("None")

In [98]:
#read cleaned txt data from qianchu
# Path to the directory containing the txt files
directory = "/content/drive/MyDrive/DS_PROJ_MAST90106/data/Cleaned txt data"

# Get a list of all the txt files in the directory
txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
accessed_file = 0
counter = 0
# Loop over each txt file and process it
for file_name in txt_files:
    accessed_file +=1
    (before,after) = None,None 
    file_path = os.path.join(directory, file_name)
    # Open the file and read its contents
    with open(file_path, 'r') as file:
        found = []
        file_contents = file.read()
        file_name = file_name.replace(".txt",".pdf")
        #print(file_contents)
        try:
          before,after =extract(file_contents)
          before = clean(before)
          after = clean(after)
          app = before[-1].lower()
          res = after[0].lower()
          print("app",app,"res",res)
          for i in df.index:
            #print(i)
            if (app in df.loc[i]["applicant"].lower()) and (res in df.loc[i]["respondent"].lower()):
                if df.loc[i]["doc"] != file_name:
                    ref_file = df.loc[i]["doc"]
                    ref_ID = df.loc[i]["vcat_no"] 
                    df.loc[df["doc"] == file_name, "refer_file"] = ref_file
                    df.loc[df["doc"] == file_name, "refer_ID"] = ref_ID
                    print(i)
                    counter+=1
        except TypeError:
          pass
        except IndexError:
          pass
        except AttributeError:
          pass

app lawley res terrace
app carr res berriman
app anor res mcdermott
app mr res sanzaro
app codelfa res state
app detm1 res dunkel
app fund res johnson
app minchilo res ford
app temloc res errill
app ors res delatite
app geftine res ictorian
app geftine res ictorian
app bellgrove res eldridge
app land res aust-china
app avramoski res barrett
app bellgrove res eldridge
app bellgrove res eldridge
app bellgrove res eldridge
app lutis res housing
app jones res dunkel
app developments res minister
app moutidis res hgfl
app tanska res transport
app anor res laurie
app pyo res vmia
app moss res australian
app 1 res brady
app bryan res maloney
app brown res dunne.
app ors res brady
app bellgrove res zalega
app i res preferred
app ors res arundel
app associates res capannolo
app watkins res israeli
app group res sidrak
app jasen res demaio
app anor res butt
app anor res wilkie
app anor res langendoen
app fraser res mason
app ors res motalli
app constructions res fleischer
app shao res a
426
app 

In [99]:
print(counter)
print(accessed_file)

241
503


In [95]:

csv_file_path = "/content/drive/MyDrive/DS_PROJ_MAST90106/data/full/extracted.csv"
# Finally, save the DataFrame to the CSV file using the to_csv() method
with open(csv_file_path,"w",encoding = "utf-8") as f:
  df.to_csv(f)